<a href="https://colab.research.google.com/github/mayonaise01/DADS5001_Data_Product/blob/main/Data_Product_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [277]:
# import library
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import pairwise_distances_argmin
import numpy as np

# **Import Data**

In [278]:
df = pd.read_csv('https://raw.githubusercontent.com/mayonaise01/DADS5001_Data_Product/refs/heads/main/marketing_campaign.csv',sep="\t")

In [279]:
# Content
# Attributes

# People

# ID: Customer's unique identifier
# Year_Birth: Customer's birth year
# Education: Customer's education level
# Marital_Status: Customer's marital status
# Income: Customer's yearly household income
# Kidhome: Number of children in customer's household
# Teenhome: Number of teenagers in customer's household
# Dt_Customer: Date of customer's enrollment with the company
# Recency: Number of days since customer's last purchase
# Complain: 1 if the customer complained in the last 2 years, 0 otherwise
# Products

# MntWines: Amount spent on wine in last 2 years
# MntFruits: Amount spent on fruits in last 2 years
# MntMeatProducts: Amount spent on meat in last 2 years
# MntFishProducts: Amount spent on fish in last 2 years
# MntSweetProducts: Amount spent on sweets in last 2 years
# MntGoldProds: Amount spent on gold in last 2 years
# Promotion

# NumDealsPurchases: Number of purchases made with a discount
# AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
# AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
# AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
# AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
# AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
# Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
# Place

# NumWebPurchases: Number of purchases made through the company’s website
# NumCatalogPurchases: Number of purchases made using a catalogue
# NumStorePurchases: Number of purchases made directly in stores
# NumWebVisitsMonth: Number of visits to company’s website in the last month

In [280]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

# **Clean Data**

In [281]:
df = df.dropna()

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2216 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2216 non-null   int64  
 1   Year_Birth           2216 non-null   int64  
 2   Education            2216 non-null   object 
 3   Marital_Status       2216 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2216 non-null   int64  
 6   Teenhome             2216 non-null   int64  
 7   Dt_Customer          2216 non-null   object 
 8   Recency              2216 non-null   int64  
 9   MntWines             2216 non-null   int64  
 10  MntFruits            2216 non-null   int64  
 11  MntMeatProducts      2216 non-null   int64  
 12  MntFishProducts      2216 non-null   int64  
 13  MntSweetProducts     2216 non-null   int64  
 14  MntGoldProds         2216 non-null   int64  
 15  NumDealsPurchases    2216 non-null   int64 

# **Explore Data**

In [283]:
df['Age'] = 2024 - df['Year_Birth']

In [284]:
# def ตรวจสอบ outlier
def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = data[(data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))]
    return outliers

In [285]:
# ตรวจสอบ outlier ที่ไม่ใช่ object
for feature in df.columns[2:]:
    if df.dtypes[feature] == 'object':
        continue
    else:
        outliers = detect_outliers_iqr(df[feature])
        print(f"Outliers in {feature}: {len(outliers)}")
        print(outliers)
        fig = px.box(df, y=feature)
        fig.show()

Outliers in Income: 8
164     157243.0
617     162397.0
655     153924.0
687     160803.0
1300    157733.0
1653    157146.0
2132    156924.0
2233    666666.0
Name: Income, dtype: float64


Outliers in Kidhome: 0
Series([], Name: Kidhome, dtype: int64)


Outliers in Teenhome: 0
Series([], Name: Teenhome, dtype: int64)


Outliers in Recency: 0
Series([], Name: Recency, dtype: int64)


Outliers in MntWines: 35
111     1332
161     1349
295     1241
424     1285
430     1248
466     1239
497     1396
515     1288
523     1379
543     1478
559     1492
824     1492
826     1279
870     1308
917     1478
937     1253
987     1394
990     1296
1001    1285
1010    1230
1052    1315
1191    1298
1458    1302
1488    1449
1492    1259
1577    1252
1641    1459
1749    1493
1922    1324
1953    1285
1961    1462
1992    1276
2067    1245
2098    1486
2127    1311
Name: MntWines, dtype: int64


...

Outliers in Complain: 21
78      1
147     1
192     1
271     1
355     1
358     1
590     1
972     1
1041    1
1069    1
1180    1
1201    1
1211    1
1277    1
1323    1
1461    1
1571    1
1720    1
1850    1
1924    1
2018    1
Name: Complain, dtype: int64


Outliers in Z_CostContact: 0
Series([], Name: Z_CostContact, dtype: int64)


Outliers in Z_Revenue: 0
Series([], Name: Z_Revenue, dtype: int64)


Outliers in Response: 333
0       1
8       1
15      1
33      1
39      1
       ..
2194    1
2198    1
2202    1
2221    1
2239    1
Name: Response, Length: 333, dtype: int64


Outliers in Age: 3
192    124
239    131
339    125
Name: Age, dtype: int64


In [286]:
# ดู outlier

outliers_income = detect_outliers_iqr(df['Income'])
print("Outliers in Income:", outliers_income)

fig = px.box(df, y="Income")
fig.show()

Outliers in Income: 164     157243.0
617     162397.0
655     153924.0
687     160803.0
1300    157733.0
1653    157146.0
2132    156924.0
2233    666666.0
Name: Income, dtype: float64


In [287]:
# ตรวจสอบ ข้อมูลที่เป็น object

for feature in df.columns:
    if df.dtypes[feature] == 'object':
      fig = px.histogram(df, x=feature)
      fig.show()

# **Remove Outlier and Grouping Data**

In [288]:
# delete row ที่มี income มากกว่า 200000

df = df[df["Income"] <= 200000]

In [289]:
# Group by Marutal Status
df['Marital_Status'] = df['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'Absurd', 'YOLO'], 'Single')
df['Marital_Status'] = df['Marital_Status'].replace(['Married', 'Together'], 'Couple')

print(df['Marital_Status'].unique())

['Single' 'Couple']


<ipython-input-289-f76d4c7b61e8>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-289-f76d4c7b61e8>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [290]:
# Group by Education

df['Education'] = df['Education'].replace(['PhD','Master'], 'Postgraduate')
df['Education'] = df['Education'].replace(['Basic'], 'Undergraduate')
df['Education'] = df['Education'].replace(['Graduation','2n Cycle'], 'Graduate')

print(df['Education'].unique())

['Graduate' 'Postgraduate' 'Undergraduate']


<ipython-input-290-8567861627cf>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-290-8567861627cf>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-290-8567861627cf>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [291]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2215 entries, 0 to 2239
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2215 non-null   int64  
 1   Year_Birth           2215 non-null   int64  
 2   Education            2215 non-null   object 
 3   Marital_Status       2215 non-null   object 
 4   Income               2215 non-null   float64
 5   Kidhome              2215 non-null   int64  
 6   Teenhome             2215 non-null   int64  
 7   Dt_Customer          2215 non-null   object 
 8   Recency              2215 non-null   int64  
 9   MntWines             2215 non-null   int64  
 10  MntFruits            2215 non-null   int64  
 11  MntMeatProducts      2215 non-null   int64  
 12  MntFishProducts      2215 non-null   int64  
 13  MntSweetProducts     2215 non-null   int64  
 14  MntGoldProds         2215 non-null   int64  
 15  NumDealsPurchases    2215 non-null   int64 

In [292]:
# One-hot encode 'Education' and 'Marital_Status'
df = pd.get_dummies(df, columns=['Education', 'Marital_Status'], drop_first=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2215 entries, 0 to 2239
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       2215 non-null   int64  
 1   Year_Birth               2215 non-null   int64  
 2   Income                   2215 non-null   float64
 3   Kidhome                  2215 non-null   int64  
 4   Teenhome                 2215 non-null   int64  
 5   Dt_Customer              2215 non-null   object 
 6   Recency                  2215 non-null   int64  
 7   MntWines                 2215 non-null   int64  
 8   MntFruits                2215 non-null   int64  
 9   MntMeatProducts          2215 non-null   int64  
 10  MntFishProducts          2215 non-null   int64  
 11  MntSweetProducts         2215 non-null   int64  
 12  MntGoldProds             2215 non-null   int64  
 13  NumDealsPurchases        2215 non-null   int64  
 14  NumWebPurchases          2215

In [293]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11] # 1<clusters< m

In [294]:
# Select features for K-means clustering
features = ['Income', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
X = df[features]

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Determine the optimal number of clusters using the elbow method
inertia = []
for i in range_n_clusters:
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# สร้างกราฟ Elbow Method
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range_n_clusters),  # แกน X เป็นจำนวน clusters
    y=inertia,  # แกน Y เป็น inertia
    mode='lines+markers',  # เส้นพร้อม marker
    marker=dict(size=10, color='blue'),  # ปรับขนาดและสี marker
    line=dict(color='blue', width=2)  # ปรับสีและความหนาเส้น
))

# เพิ่มชื่อแกนและชื่อกราฟ
fig.update_layout(
    title="Elbow Method for Optimal k",
    xaxis_title="Number of clusters",
    yaxis_title="Inertia",
    template="plotly_white"  # ธีมแบบขาวสะอาด
)

# แสดงกราฟ
fig.show()

In [295]:
# silhouette analysis

for k in range_n_clusters:
  clusterer = KMeans(n_clusters=k, random_state=10)
  cluster_labels = clusterer.fit_predict(X_scaled)
  silhouette_avg = silhouette_score(X_scaled, cluster_labels)
  print(
      "For n_clusters =",
      k,
      "The average silhouette_score is :",
      silhouette_avg,
  )

For n_clusters = 2 The average silhouette_score is : 0.4798863529328558
For n_clusters = 3 The average silhouette_score is : 0.39948505832785725
For n_clusters = 4 The average silhouette_score is : 0.4036900927743438
For n_clusters = 5 The average silhouette_score is : 0.34789789462576026
For n_clusters = 6 The average silhouette_score is : 0.3117617683987502
For n_clusters = 7 The average silhouette_score is : 0.29705461992352494
For n_clusters = 8 The average silhouette_score is : 0.2834642112917925
For n_clusters = 9 The average silhouette_score is : 0.2730454216370531
For n_clusters = 10 The average silhouette_score is : 0.27475315278712065
For n_clusters = 11 The average silhouette_score is : 0.27296612354508476


In [296]:
# Based on the elbow method, choose the optimal number of clusters (e.g., k=2)
k = 2
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(X_scaled)

# Add cluster labels to the DataFrame
df['Cluster'] = kmeans.labels_

# Analyze the clusters
for i in range(k):
    print(f"\nCluster {i}:")
    print(df[df['Cluster'] == i][features].describe())

# Visualize the clusters (you can use different features for visualization)
fig = px.scatter(
    df,
    x='Income',  # แกน X
    y='MntWines',  # แกน Y
    color='Cluster',  # สีของจุดตามคลัสเตอร์
    color_continuous_scale='viridis',  # แผนสีเดียวกับ matplotlib
    title='K-means Clustering Results',
    labels={'Income': 'Income', 'MntWines': 'Amount spent on Wines'},  # ชื่อแกน
)

# ปรับแต่งเพิ่มเติม เช่น ขนาดกราฟ
fig.update_layout(
    xaxis_title='Income',
    yaxis_title='Amount spent on Wines',
    template='plotly_white'  # ธีมสะอาด
)

# แสดงกราฟ
fig.show()


Cluster 0:
             Income     MntWines    MntFruits  MntMeatProducts  \
count   1217.000000  1217.000000  1217.000000      1217.000000   
mean   36824.602301    76.815941     5.816763        31.465900   
std    13000.189044   104.375897     8.838748        34.669099   
min     1730.000000     0.000000     0.000000         0.000000   
25%    27242.000000    10.000000     1.000000         9.000000   
50%    36959.000000    29.000000     3.000000        18.000000   
75%    46098.000000   100.000000     7.000000        42.000000   
max    73395.000000   582.000000    67.000000       226.000000   

       MntFishProducts  MntSweetProducts  MntGoldProds  
count      1217.000000       1217.000000   1217.000000  
mean          8.764174          6.020542     18.191454  
std          14.470061          9.961294     23.455031  
min           0.000000          0.000000      0.000000  
25%           2.000000          0.000000      5.000000  
50%           4.000000          3.000000     11.000

In [297]:
# Select features1 for K-means clustering
features1 = ['Income', 'Education_Postgraduate','Education_Undergraduate','Marital_Status_Single','Recency','NumDealsPurchases','AcceptedCmp1']
X = df[features1]

# Standardize the features
scaler = StandardScaler()
X_scaled1 = scaler.fit_transform(X)

# Determine the optimal number of clusters using the elbow method
inertia = []
for i in range_n_clusters:
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(X_scaled1)
    inertia.append(kmeans.inertia_)

# สร้างกราฟ Elbow Method
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range_n_clusters),  # แกน X เป็นจำนวน clusters
    y=inertia,  # แกน Y เป็น inertia
    mode='lines+markers',  # เส้นพร้อม marker
    marker=dict(size=10, color='blue'),  # ปรับขนาดและสี marker
    line=dict(color='blue', width=2)  # ปรับสีและความหนาเส้น
))

# เพิ่มชื่อแกนและชื่อกราฟ
fig.update_layout(
    title="Elbow Method for Optimal k",
    xaxis_title="Number of clusters",
    yaxis_title="Inertia",
    template="plotly_white"  # ธีมแบบขาวสะอาด
)

# แสดงกราฟ
fig.show()

In [298]:
# silhouette analysis

for k in range_n_clusters:
  clusterer = KMeans(n_clusters=k, random_state=10)
  cluster_labels = clusterer.fit_predict(X_scaled1)
  silhouette_avg = silhouette_score(X_scaled1, cluster_labels)
  print(
      "For n_clusters =",
      k,
      "The average silhouette_score is :",
      silhouette_avg,
  )

For n_clusters = 2 The average silhouette_score is : 0.20989382951225355
For n_clusters = 3 The average silhouette_score is : 0.1835509212833631
For n_clusters = 4 The average silhouette_score is : 0.22966532685175967
For n_clusters = 5 The average silhouette_score is : 0.2641440126510871
For n_clusters = 6 The average silhouette_score is : 0.2575477335050753
For n_clusters = 7 The average silhouette_score is : 0.3267454235321183
For n_clusters = 8 The average silhouette_score is : 0.27630020936057786
For n_clusters = 9 The average silhouette_score is : 0.29546058028165545
For n_clusters = 10 The average silhouette_score is : 0.30007629397553354
For n_clusters = 11 The average silhouette_score is : 0.3024871984499595


In [299]:
# Based on the elbow method, choose the optimal number of clusters (e.g., k=7)
k = 7
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(X_scaled1)

# Add cluster labels to the DataFrame
df['Cluster'] = kmeans.labels_

# Analyze the clusters
for i in range(k):
    print(f"\nCluster {i}:")
    print(df[df['Cluster'] == i][features1].describe())


Cluster 0:
              Income     Recency  NumDealsPurchases  AcceptedCmp1
count     177.000000  177.000000         177.000000         177.0
mean    52058.508475   48.316384           7.107345           0.0
std     17339.007276   27.830686           2.394312           0.0
min      1730.000000    0.000000           5.000000           0.0
25%     44010.000000   24.000000           6.000000           0.0
50%     52278.000000   51.000000           6.000000           0.0
75%     58710.000000   70.000000           7.000000           0.0
max    160803.000000   98.000000          15.000000           0.0

Cluster 1:
              Income     Recency  NumDealsPurchases  AcceptedCmp1
count     476.000000  476.000000         476.000000         476.0
mean    53661.651261   48.735294           1.976891           0.0
std     20112.157711   28.635151           1.216333           0.0
min      5648.000000    0.000000           0.000000           0.0
25%     37769.500000   23.000000           1.000000 

In [303]:
# prompt: จาก cluster ทั้ง 7 อันด้านบน อยากลองแบ่งดูด้วยว่า income เป็นคนมีรายได้ สูง กลาง ต่ำ หรือ AcceptedCmp1 มีคนซื้ออันนี้กี่อันแบบ สูง กลาง ต่ำ เพื่อให้คนอ่านเข้าใจ conclusion แนะนำ code หน่อย

import pandas as pd
import plotly.express as px

# ... (your existing code)

# Based on the elbow method, choose the optimal number of clusters (e.g., k=7)
k = 7
kmeans = KMeans(n_clusters=k, random_state=0)
kmeans.fit(X_scaled1)

# Add cluster labels to the DataFrame
df['Cluster'] = kmeans.labels_

# Analyze the clusters and categorize income and AcceptedCmp1
for i in range(k):
    cluster_data = df[df['Cluster'] == i]
    income_median = cluster_data['Income'].median()
    accepted_cmp1_count = cluster_data['AcceptedCmp1'].count()

    # Categorize income
    if income_median > 100000:
        income_category = 'สูง'
    elif income_median > 50000:
        income_category = 'กลาง'
    else:
        income_category = 'ต่ำ'

    #Categorize AcceptedCmp1
    if accepted_cmp1_count > 400:
      accepted_cmp1_category = "สูง"
    elif accepted_cmp1_count > 100:
      accepted_cmp1_category = "กลาง"
    else:
      accepted_cmp1_category = "ต่ำ"

    print(f"\nCluster {i}:")
    print(f"  Income Category: {income_category}")
    print(f"  AcceptedCmp1 Category: {accepted_cmp1_category}")
    print(cluster_data[features1].describe())

# ... (rest of your code)


Cluster 0:
  Income Category: กลาง
  AcceptedCmp1 Category: กลาง
              Income     Recency  NumDealsPurchases  AcceptedCmp1
count     177.000000  177.000000         177.000000         177.0
mean    52058.508475   48.316384           7.107345           0.0
std     17339.007276   27.830686           2.394312           0.0
min      1730.000000    0.000000           5.000000           0.0
25%     44010.000000   24.000000           6.000000           0.0
50%     52278.000000   51.000000           6.000000           0.0
75%     58710.000000   70.000000           7.000000           0.0
max    160803.000000   98.000000          15.000000           0.0

Cluster 1:
  Income Category: กลาง
  AcceptedCmp1 Category: สูง
              Income     Recency  NumDealsPurchases  AcceptedCmp1
count     476.000000  476.000000         476.000000         476.0
mean    53661.651261   48.735294           1.976891           0.0
std     20112.157711   28.635151           1.216333           0.0
min      56